In [1]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om
    
from openmdao.utils.assert_utils import assert_near_equal

# Solver Recording
Solver recording is useful when you want to record the iterations within a solver. 
The recorder can capture the values of states, errors, and residuals as the solver converges.

In [2]:
om.show_options_table("openmdao.solvers.solver.Solver", recording_options=True)

Option,Default,Acceptable Values,Acceptable Types,Description
excludes,[],N/A,['list'],Patterns for vars to exclude in recording. (processed post-includes) Paths are relative to solver's Group. Uses fnmatch wildcards
includes,['*'],N/A,['list'],Patterns for variables to include in recording. Paths are relative to solver's Group. Uses fnmatch wildcards
record_abs_error,True,"[True, False]",['bool'],Set to True to record absolute error at the solver level
record_inputs,True,"[True, False]",['bool'],Set to True to record inputs at the solver level
record_metadata,True,"[True, False]",['bool'],Deprecated. Recording of metadata will always be done
record_outputs,True,"[True, False]",['bool'],Set to True to record outputs at the solver level
record_rel_error,True,"[True, False]",['bool'],Set to True to record relative error at the solver level
record_solver_residuals,False,"[True, False]",['bool'],Set to True to record residuals at the solver level


> **_Note:_**  Note that the `excludes` option takes precedence over the `includes` option.

> **_Note:_**  The paths given in the `includes` and `excludes` options are relative to the `Group` that the solver is attached to.

> **_Note:_**  It is currently not possible to record linear solvers.

## Solver Recording Example

In [3]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
prob.setup()

recorder = om.SqliteRecorder("cases.sql")

solver = prob.model.nonlinear_solver
solver.add_recorder(recorder)
solver.recording_options['record_abs_error'] = True
solver.options['use_apply_nonlinear'] = True

prob.run_model()
prob.cleanup()

NL: NLBGS Converged in 7 iterations


In [4]:
cr = om.CaseReader("cases.sql")
solver_cases = cr.list_cases('root.nonlinear_solver')

solver
rank0:root._solve_nonlinear|0|NonlinearBlockGS|0


solver
rank0:root._solve_nonlinear|0|NonlinearBlockGS|1


solver
rank0:root._solve_nonlinear|0|NonlinearBlockGS|2


solver
rank0:root._solve_nonlinear|0|NonlinearBlockGS|3


solver
rank0:root._solve_nonlinear|0|NonlinearBlockGS|4


solver
rank0:root._solve_nonlinear|0|NonlinearBlockGS|5


solver
rank0:root._solve_nonlinear|0|NonlinearBlockGS|6


In [5]:
assert len(solver_cases) == 7

In [6]:
for case_id in solver_cases:
    print(cr.get_case(case_id).abs_err)

2.2545141061171243
0.043663258684468076
0.0008635223885384846
1.707062189737485e-05
3.3746495375908125e-07
6.671260877055829e-09
1.318802844707534e-10


In [7]:
assert_near_equal(cr.get_case(0).abs_err, 2.2545141, tolerance=1e-8)

2.7132782804768756e-09